# Camera calibration

In [1]:
#importing the required modules
import numpy as np
import cv2
import glob
import os

In [2]:
# termination criteria for refinement of corners
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Defining the world coordinates for 3D points
objp = np.zeros((9*12,3), np.float32)
#since the size of the sqaures is 1cm =10mm thus, prepare object points like (0,0,0), (10,0,0), (20,0,0)...
objp[:,:2] = np.mgrid[0:90:10,0:120:10].T.reshape(-1,2)



# Creating vector to store vectors of 3D pointsand 2D points for each checkerboard image
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

# Extracting path of individual image stored in a given directory
images = glob.glob('/Users/jainparag/Desktop/Camera Calibration/Images/*.bmp')
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true

    ret, corners = cv2.findChessboardCorners(gray, (9,12),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,12), corners2,ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)
        cv2.destroyAllWindows()



In [3]:

#Performing camera calibration by assing the value of known 3D points(objpoints)
#and corresponding pixel coordinates of the detected corners (imgpoints)

ret, mtx,  dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)


print(ret)
print(mtx)
print(dist)
print(rvecs)
print(tvecs)

0.23207594510939056
[[4.73548337e+03 0.00000000e+00 1.05055561e+03]
 [0.00000000e+00 4.73293515e+03 8.20417716e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[-1.17971690e-01  7.31321938e-02  6.87719439e-04 -5.69837404e-04
  -1.86385341e+00]]
[array([[ 0.02829248],
       [ 0.02886229],
       [-1.56546146]]), array([[ 0.06988597],
       [-0.10232018],
       [-2.64276517]]), array([[ 9.28791350e-03],
       [ 9.67323528e-05],
       [-1.58096178e+00]]), array([[ 0.02992548],
       [ 0.03000645],
       [-1.60616079]]), array([[-0.04800625],
       [ 0.39056751],
       [-2.65253438]]), array([[-0.00380466],
       [-0.24262436],
       [-2.66993582]]), array([[ 0.03143881],
       [ 0.04111383],
       [-1.57323288]]), array([[ 0.02596282],
       [ 0.03629974],
       [-1.58802198]]), array([[ 0.02982208],
       [ 0.03071858],
       [-1.55793723]]), array([[ 0.03256938],
       [ 0.01191527],
       [-0.80063427]]), array([[-0.20622828],
       [-0.23028477],
       [-1.5

# Additional

In [4]:
# Undistorting the images 
for i in images:
    img = cv2.imread(i)
    h,  w = img.shape[:2]
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
    # undistort
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

    # crop the image
    x,y,w,h = roi
    dst = dst[y:y+h, x:x+w]
    
    head = os.path.basename(str(i))
    head,_= os.path.splitext(head)
    cv2.imwrite('/Users/jainparag/Desktop/Camera Calibration/Undistored images/{}_undistored.png'.format(head),dst)
    

In [5]:
#Re-projection Error
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error

print ("total error: ", mean_error/len(objpoints))

total error:  0.02211154882881284
